In [10]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from scipy import interpolate
import biosig as bs
import os
import _cut_signal as ct

def open_files(path):
    files = [file for file in os.listdir(path) if file[-4:] in ('.WFT', '.wft')]
    signals = pd.DataFrame({'time': [],
                        'incid':[],
                        'reflect':[]})

    for file in files:
        HDR=bs.header(path+file)
        sr = float(HDR.split('\n')[9][HDR.split('\n')[9].find(':')+2:-1])
        data=bs.data(path+file)


        res = []
        for item in data:
            res.append(item[0])
        if signals.incid.sum() == 0:
            signals.incid = res
        elif signals.incid.max() > max(res):
            signals.reflect = res
        else:
            signals.reflect = signals.incid
            signals.incid = res
    signals.time = np.arange(len(data))/sr

    return signals
    



In [11]:
way = '/Users/lubimyj/Git/experiments/Plansee-Tungstan-W/'
materials = ['Denal920','DX2']
specimens = ['1','2','3','4','5','6']
to_IR_data = '/Specimen'
to_raw_data = '/test/Original WFT Files/'
IR_index = '_IR/'
subFold = '/IRCamera/Specimen'
testFold = 'test/EXP #1/'
dataFold = '/FLT/'

material = materials[1]
specimen = specimens[5]



In [12]:
mech_data_path = way + material + subFold + specimen + '/'
raw_data_path = way + material + subFold + specimen + to_raw_data
thermo_data_path = way + material + subFold + specimen + to_IR_data + specimen + IR_index
mech_data = pd.read_csv(mech_data_path+'result_data.txt', sep='\t')
#thermo_data = pd.read_csv(thermo_data_path +'Temperature_Spec_'+specimen+'.csv', sep = '\t' )           ##Treshhold by MIN
thermo_data = pd.read_csv(thermo_data_path +'Temperature_Spec_'+specimen+'Max_Tr.csv', sep = '\t' )     ##Treshhold by MAX
raw_data = open_files(raw_data_path)
title = 'Tr_0.9'




In [13]:
fig1 = go.Figure()
fig1 = make_subplots(specs=[[{"secondary_y": True}]])
fig1.update_layout(template = 'plotly_white', width=800, height=600,
                  title = f'Comparison IncidentWave vs Temperature, {material} specimen {specimen} {title}')

fig1.add_trace(go.Scatter(x = thermo_data.time-107e-6, 
                            y = thermo_data[title], 
                            name = 'Temperature, C'),
                            secondary_y=True)
fig1.add_trace(go.Scatter(x = raw_data.time, 
                            y = raw_data.incid, 
                            name = 'Raw'))

fig1.update_xaxes(title_text="Time, s")

fig1.update_yaxes(title_text="Voltage", secondary_y=False)
fig1.update_yaxes(title_text="Temperature", secondary_y=True, range=[-thermo_data[title].max()-10,thermo_data[title].max()+10])

fig1.show()

In [16]:
#func_sign, func_time = cut_signal(mech_data.true_strain, thermo_data[title], mech_data.time, thermo_data.time)
#func_sign, func_time = cut_signal(thermo_data[title], thermo_data.time)
temperature = ct.cut_signal(mech_data.true_strain, thermo_data[title], mech_data.time, thermo_data.time)
#print(len(func_sign), len(mech_data.true_strain))


In [17]:
#fig1 = go.Figure()
fig1 = make_subplots(specs=[[{"secondary_y": True}]])
fig1.update_layout(template = 'plotly_white', width=800, height=600,
                  title = 'Comparison Strain vs Temperature')
fig1.add_trace(go.Scatter(x = mech_data.time, 
                            y = mech_data.true_strain,
                            name = 'True Strain'),
                            secondary_y=False)
fig1.add_trace(go.Scatter(x = mech_data.time, 
                            y = mech_data.true_stress,
                            name = 'True Stress'),
                            secondary_y=False)
fig1.add_trace(go.Scatter(x = temperature.time, 
                            y = temperature.value, 
                            name = 'Temperature'),
                            secondary_y=True)

fig1.update_xaxes(title_text="Time, s")

fig1.update_yaxes(title_text="Strain", secondary_y=False)
fig1.update_yaxes(title_text="Temperature", secondary_y=True)

fig1
fig1.show()

In [18]:
fig2 = make_subplots(specs=[[{"secondary_y": True}]])
fig2.update_layout(template = 'plotly_white', width=800, height=600,
                  title = f'True Strain vs Temperature, {material} specimen {specimen}')
fig2.add_trace(go.Scatter(x = mech_data.true_strain, 
                            y = mech_data.true_stress,
                            name = 'True Stress'),
                            secondary_y=False)
fig2.add_trace(go.Scatter(x = mech_data.true_strain, 
                            y = temperature.value, 
                            name = 'Temperature'),
                            secondary_y=True)

fig2.update_xaxes(title_text="Strain")

fig2.update_yaxes(title_text="Stress, MPa", secondary_y=False)
fig2.update_yaxes(title_text="Temperature, C", secondary_y=True)

fig2
fig2.show()

In [19]:
fig3 = go.Figure()
fig3.update_layout(template = 'plotly_white', width=800, height=600,
                  title = f'True Strain vs Temperature, {material} specimen {specimen}')
fig3.add_trace(go.Scatter(x = mech_data.true_strain, 
                            y = temperature.value/mech_data.true_stress, 
                            name = 'Temperature'))

fig3.update_xaxes(title_text="Strain")

fig3.update_yaxes(title_text="Betha")

fig3
fig3.show()